In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# URL del listado de productos
url = 'https://listado.mercadolibre.com.pe/carros#D[A:carros]'

# Encabezados para la solicitud
encabezados = {
    "User -Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Realizar la solicitud
response = requests.get(url, headers=encabezados)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

# Verificar si la conexión es exitosa
if response.status_code == 200:
    print("Conexión exitosa con la página")
else:
    print("Error al conectar con la página")

# Listas para almacenar datos
nombres = []
precios = []

# Extraer productos
productos = soup.find_all('h2', class_='poly-box poly-component__title')

for producto in productos:
    # Extraer el nombre
    nombre = producto.find('a')
    if nombre:
        nombres.append(nombre.text.strip())
    
    # Extraer el precio
    precio_div = producto.find_next('div', class_='poly-component__price')
    if precio_div:
        precio_span = precio_div.find('span', class_='andes-money-amount andes-money-amount--cents-superscript')
        if precio_span:
            precio = precio_span.find('span', class_='andes-money-amount__fraction')
            if precio:
                precios.append(precio.text.strip())

# Verificar las longitudes de las listas
print(f"Productos encontrados: {len(nombres)}")
print(f"Precios encontrados: {len(precios)}")

# Asegurarse de que las listas tengan la misma longitud
min_length = min(len(nombres), len(precios))
nombres = nombres[:min_length]
precios = precios[:min_length]

# Crear DataFrame
df_total = pd.DataFrame({
    'Nombre Producto': nombres,
    'Precio': precios
})


# Imprimir resultados
print(df_total)

Conexión exitosa con la página
Productos encontrados: 48
Precios encontrados: 48
                                    Nombre Producto  Precio
0               Citroën C'elysee 1.6i Bvm Seduction   3.000
1                                       Kia Rio 1.4  10.900
2                          Mazda Cx-9 Signature Awd  40.990
3                       Toyota Hilux Sw4 Ambulancia   5.000
4                    Jeep Cherokee Limited 2.4 Full  16.500
5                            Nissan Tiida Hatchback  24.700
6                             Nissan Tiida Sedan 16   6.550
7               Mazda A3 Sedan Mazda 3 Sedan Mt 2.0  11.200
8                                      Bmw 525i 2.5  11.900
9                         Jac T8 Comfort Diésel 4x4  16.600
10                             Hyundai Santa Fe 4x2      35
11                 Renault Fluence Dinamique 2.0 At  13.900
12                         Mazda Captiva Cx-9 Mazda  13.900
13  Porsche Cayenne S Tiptronic Cayenne S Tiptronic  29.900
14                 